In [1]:
import pandas as pd

**Part 1** Here I'm reading in the SEP data

In [32]:
#read in the Balch SEP event list
sepdata = pd.read_excel("SPEall.v7p.xls")

In [33]:
#read in the Balch SEP event list
ctrldata = pd.read_excel("ctrlevents.v8p.xls")

In [34]:
list(sepdata)

['Onset',
 'Threshold',
 'Maxtime',
 'Maxflux',
 'Endtime',
 'EPSSatId',
 'FlrOnset',
 'Flrmaxtime',
 'Flrendtime',
 'FlrPeakFlux',
 'xrscls',
 'ocls',
 'optlocation',
 'region',
 'TypeII',
 'TII_Duration',
 'TypeIV',
 'TIV_duration',
 'CME_speed',
 'XRSSatId',
 'FlrHpTime',
 'FlrIntFlux',
 'FlrIntFlux2',
 'P10MAX',
 'P30MAX',
 'P60MAX',
 'P100MAX',
 'P10FLUENCE',
 'P30FLUENCE',
 'P60FLUENCE',
 'P100FLUENCE']

In [37]:
pd.plotting.scatter_matrix(sepdata, alpha=0.2, figsize=(4,4), diagonal='hist', \
                                 columns=['Maxflux','FlrPeakFlux','TII_Duration','TIV_duration']) #,\
                                          #'FlrIntFlux','FlrIntFlux2','P10MAX','P30MAX','P60MAX']) #,\
                                          #'P100MAX','P10FLUENCE','P30FLUENCE','P60FLUENCE','P100FLUENCE'])

AttributeError: Unknown property columns